# Aplicação para criar tabela fato de atestados e afastaos

Esta aplicação pegas os dados da tabela de afastados e atestados e os organizam de forma que consiga em uma tabela fato de maeira que se consiga saber o numero de dias perdidos por mês

In [1]:
import csv
import xlrd
from tkinter import *
from functools import partial
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd
from datetime import datetime, timedelta

In [2]:
#---------------------------------------------------------
#    Função abre caixa de dialogo para localizar aquivo
#---------------------------------------------------------
def fun_Caminho():
        #---Abre a caixa de dialogo e atribui o caminho a string ---------
            caminho = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("Excel files","*.xls*"),("all files","*.*")))
            return caminho
        

In [3]:
#--------------------------------------------------------------------
#    Carrega as informações do arquivo para dentro de uma lista
#--------------------------------------------------------------------

strCaminho = fun_Caminho() # Chama a função que localiza o caminho do arquivo

#----------Verifica se algum arquivo foi selecionado--------------------------
if strCaminho !='':
    tabela = xlrd.open_workbook(strCaminho).sheet_by_index(0)
    qtd_linhas = tabela.nrows
    dAtestados=[]
    for i in range(1, qtd_linhas):
        dAtestados.append(tabela.row_values(i))
mainloop()
        
    

In [4]:
#---------------------------------------------------------------------
#           Carrega o DataFrame com as informações do arquivo
#--------------------------------------------------------------------

df=pd.DataFrame(dAtestados) #--Carega o DataFrame
df.columns=[tabela.row_values(0)]

#--------------Renomeio a coluna ce inicio e fim do atestado---------------
df.rename(columns={'Início Atestado':'dtInicio'},inplace=True)
df.rename(columns={'Término Atestado':'dtFim'},inplace=True)
df.rename(columns={'Ficha Médica':'Ficha'},inplace=True)

In [5]:
#------------------------------------------------------------------------
#           Cria tabela calendario auxiliar
#------------------------------------------------------------------------
dtInicio = datetime.strptime(df.dtInicio.min(), '%d/%m/%Y')-timedelta(days=1)
dtFim = datetime.strptime(df.dtFim.max(), '%d/%m/%Y')+timedelta(days=1)
nDias= abs((dtFim-dtInicio).days)

dCalendario=[]
for dtBase in range(1, nDias):
    dCalendario.append(dtInicio + timedelta(days=dtBase))
    

In [6]:
#-----------------Cria um DataFrame de calendario -----------------
dfCalendario=pd.DataFrame(dCalendario)
dfCalendario.columns=['dtBase']

#------Converte as colunas em datatime ------------------------
df['dtInicio']=pd.to_datetime(df['dtInicio'])
df['dtFim']=pd.to_datetime(df['dtFim'])

In [7]:
#--------------------------------------------------------------------------
#           Cria a tabela fato Atestado com as datas dia a dia
#--------------------------------------------------------------------------
lista=[]
for row1 in dfCalendario.itertuples():
    for row2 in df.itertuples():
       if row1.dtBase >= row2.dtInicio and row1.dtBase<=row2.dtFim:
        #print(row1.dtBase, row2.dtInicio, row2.dtFim) 
            lista.append([row1.dtBase, row2.dtInicio, row2.dtFim,row2.CID,row2.Empresa,row2.Ficha])
       

In [13]:
#----------------Carrega o DataFrame com a lista da tabeal fato --------------------
dfAtest=pd.DataFrame(lista)
dfAtest.columns=['dtBase','dtInicio','dtFim','CID','Empresa','Ficha']
#dfAtest.sort_values(by=['Ficha','dtInicio'])

In [12]:
#-----------------Salva a tabela em um arquivo CSV ---------------------------------------
dfAtest.to_csv(r'C:\Users\marcos.souto\Desktop\Diversos\fAtestados.csv',sep=',')
